### Our goal: create a neural network from scratch

In [119]:
import pandas as pd
import numpy as np
import torch
from torchvision.datasets import MNIST
import math
from functools import partial

### Get the data

In [ ]:
trainset = MNIST('', download=True, train=True)
testset = MNIST('', download=True, train=False)

In [43]:
# 60 000 / 10 000: classic MNIST
trainset.data.shape, testset.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [83]:
train = trainset.data.reshape(60000, -1)
test = testset.data.reshape(10000, -1)

In [160]:
y_train = trainset.targets
y_test = testset.targets

In [84]:
# FLoating point numbers work better with pytorch...
train.dtype

torch.uint8

In [85]:
train = train.to(torch.float32)
test = test.to(torch.float32)

In [86]:
# The data isn't normalized
m, std = train.mean(), train.std()
m, std

(tensor(33.3184), tensor(78.5675))

In [87]:
train = (train - m) / std
test = (test - m) / std

### define a fully connected neural net

In [110]:
def linear(x, neurons=50):
    w = torch.randn(x.shape[1], neurons) * math.sqrt(2/m)
    b = torch.randn(neurons)
    return x @ w + b

In [117]:
def relu(x):
    return x.clamp_min(0.)

In [122]:
def model(x):
    x = linear(x, 50)
    x = relu(x)
    x = linear(x, 10)
    return x

In [123]:
output = model(train)

In [125]:
#ten predictions for every row, sounds good
output.shape

torch.Size([60000, 10])

### define a loss function

In [129]:
my_tens = torch.tensor([3, 2,1, 0.5, -1])

In [145]:
def log_softmax(output):
    return (output.exp() / output.exp().sum(-1, keepdim=True)).log()

### softmax(my_tens)

In [151]:
# log-sum-exp-trick:          https://www.xarg.org/2016/06/the-log-sum-exp-trick-in-machine-learning/
def logsumexp(x):
    m = x.max(-1)[0]
    return m + (x-m[:,None]).exp().sum(-1).log()

def log_softmax(x): 
    return x - x.logsumexp(-1,keepdim=True)

In [174]:
def nll(output, y_train):
    return -output[range(y_train.shape[0]), y_train].mean()

In [175]:
nll(log_softmax(output), y_train)

tensor(10.4309)

### Forward pass

In [178]:
output = model(train)
print('output shape:', output.shape)
loss = nll(log_softmax(output), y_train)
print('loss', loss)

output shape: torch.Size([60000, 10])
loss tensor(12.3455)


### Backward pass

In [311]:
# parameters we can change:
EPOCHS = 35
INPUT = train
TARGETS = y_train
N_LAYERS = 2
LR = 0.3

# Automated fully connected neural net creation based on above parameters
activations = np.linspace(50, 10, N_LAYERS).astype(int)
shapes = [INPUT.shape[1]]
shapes.extend(activations)
w = []
b = []
for layer in range(N_LAYERS):
    w.append(torch.randn(shapes[layer], activations[layer]) * math.sqrt(2/m))
    b.append(torch.zeros(activations[layer]))
    w[-1].requires_grad_()
    b[-1].requires_grad_()
    
# Training loop
for i in range(EPOCHS):
    output = INPUT
    for layer in range(N_LAYERS-1):
        output = output @ w[layer] + b[layer]
        output = relu(output)
    
    output = output @ w[N_LAYERS-1] + b[N_LAYERS-1]
    
    loss = nll(log_softmax(output), TARGETS)
    print(loss)
    loss.backward()
    with torch.no_grad():
        for layer in range(N_LAYERS):
            w[layer] -= w[layer].grad * LR
            b[layer] -= b[layer].grad * LR
            w[layer].grad.zero_()
            b[layer].grad.zero_()

tensor(15.8638, grad_fn=<NegBackward>)
tensor(24.3053, grad_fn=<NegBackward>)
tensor(6.5728, grad_fn=<NegBackward>)
tensor(2.7249, grad_fn=<NegBackward>)
tensor(1.7518, grad_fn=<NegBackward>)
tensor(1.5279, grad_fn=<NegBackward>)
tensor(1.4192, grad_fn=<NegBackward>)
tensor(1.3349, grad_fn=<NegBackward>)
tensor(1.2638, grad_fn=<NegBackward>)
tensor(1.2018, grad_fn=<NegBackward>)
tensor(1.1478, grad_fn=<NegBackward>)
tensor(1.1002, grad_fn=<NegBackward>)
tensor(1.0581, grad_fn=<NegBackward>)
tensor(1.0207, grad_fn=<NegBackward>)
tensor(0.9874, grad_fn=<NegBackward>)
tensor(0.9573, grad_fn=<NegBackward>)
tensor(0.9299, grad_fn=<NegBackward>)
tensor(0.9050, grad_fn=<NegBackward>)
tensor(0.8820, grad_fn=<NegBackward>)
tensor(0.8609, grad_fn=<NegBackward>)
tensor(0.8411, grad_fn=<NegBackward>)
tensor(0.8228, grad_fn=<NegBackward>)
tensor(0.8056, grad_fn=<NegBackward>)
tensor(0.7894, grad_fn=<NegBackward>)
tensor(0.7742, grad_fn=<NegBackward>)
tensor(0.7598, grad_fn=<NegBackward>)
tensor(0.7

In [312]:
# even with this simple model, we see the loss decrease

### Inference

In [313]:
def accuracy(pred, targs):
    return float((pred.argmax(dim=-1) == targs).sum()) / float(len(pred))

In [314]:
output = test
for layer in range(N_LAYERS-1):
    output = output @ w[layer] + b[layer]
    output = relu(output)
    
output = output @ w[N_LAYERS-1] + b[N_LAYERS-1]
output = log_softmax(output)

In [315]:
accuracy(output, y_test)

0.7999

80% accuracy (random would be 10%,as there are 10 classes), our model is definitely learning something.
However, this code is messy and hard to maintain. In the next notebook we'll refactor it using python classes. Moreover, fully connected is a good start, but the tool of trade for computer vision is convolutions. We'll add that as well